In [74]:
import netsquid as ns
from netsquid.nodes import Node, DirectConnection
from netsquid.components import QuantumChannel, ClassicalChannel
from netsquid.protocols import NodeProtocol
from netsquid.components.models.delaymodels import FibreDelayModel
import numpy as np

In [75]:
#Define nodes in the network
n=2
source_node = Node(name="Source")
verifier_node = Node(name="Verifier")
parties_node={}
for i in range(1,n):
    parties_node["Party"+str(i)]=Node(name="Party"+str(i))

#Define channels
delay_model = FibreDelayModel()
distance = 400/1000 #Distance between nodes in km

#Quantum channel
qchannelv = QuantumChannel(name = "qchannelS2V",length = distance, models = {"delay_model":delay_model})
qchannelp={}
for i in range(1,n):
    qchannelp["Party"+str(i)] = QuantumChannel(name = "qchannelS2P"+str(i),length = distance, models = {"delay_model":delay_model})

#Classical channels
channelp={}
for i in range(1,n):
    for j in range(i,n):
            channelp["V2Party"+str(i)] = ClassicalChannel("V_to_Party"+str(i),length = distance, models = {"delay_model":delay_model})
            channelp["Party"+str(i)+"2V"] = ClassicalChannel("Party"+str(i)+"_to_V",length = distance, models = {"delay_model":delay_model})


#Add ports to nodes
source_node.add_ports(["vout"])
verifier_node.add_ports(['sin'])
for i in range(1,n):
    parties_node["Party"+str(i)].add_ports(["sin","vin","vout"])
    verifier_node.add_ports(['p'+str(i)+'in','p'+str(i)+'out'])
    source_node.add_ports(['p'+str(i)+'out'])

#Connect ports to channels
for i in range(1,n):
    parties_node["Party"+str(i)].ports['vout'].connect(channelp["Party"+str(i)+"2V"].ports['send'])
    verifier_node.ports['p'+str(i)+'in'].connect(channelp["Party"+str(i)+"2V"].ports['recv'])
    
    verifier_node.ports['p'+str(i)+'out'].connect(channelp["V2Party"+str(i)].ports['send'])
    parties_node["Party"+str(i)].ports['vin'].connect(channelp["V2Party"+str(i)].ports['recv'])
    
    source_node.ports['p'+str(i)+'out'].connect(qchannelp["Party"+str(i)].ports['send'])
    parties_node["Party"+str(i)].ports['sin'].connect(qchannelp["Party"+str(i)].ports['recv'])    

source_node.ports['vout'].connect(qchannelv.ports['send'])
verifier_node.ports['sin'].connect(qchannelv.ports['recv'])



In [76]:
class VerifierProtocol(NodeProtocol):
    def __init__(self,node, n):
        super().__init__(node)
        self.n = n
    
    #Protocol definition starts here
    def run(self):
        yield self.await_port_input(self.node.ports['sin'])
        psi = self.node.ports['sin'].rx_input().items[0]
        
        X = []
        Y = []
        
        for i in range(1,self.n):
            bit = np.random.randint(2)
            X.append(bit)
            self.node.ports['p'+str(i)+'out'].tx_output(bit)
            yield self.await_port_input(self.node.ports['p'+str(i)+'in'])
            Y.append(self.node.ports['p'+str(i)+'in'].rx_input().items[0])
            
        x=np.sum(X)%2
        X.append(x)
        #print(X)
        if x == 0:
            ns.qubits.operate(psi, ns.Z)
        elif x==1:
            ns.qubits.operate(psi, ns.H)
        m,p=ns.qubits.measure(psi)
        Y.append(m)
        #print(Y)
        print(np.sum(Y)%2==(np.sum(X)/2)%2)
        test=(np.sum(Y)%2==(np.sum(X)/2)%2)
        if(test==False):
            print('Dishonest model')
            for i in range(1,self.n):
                self.node.ports['p'+str(i)+'out'].tx_output(0) #Send failure message
            return
        for i in range(1,self.n):
                self.node.ports['p'+str(i)+'out'].tx_output(1) #Send success message
        print("Honest model")

In [77]:
class PartyProtocol(NodeProtocol):
    def __init__(self,node, n):
        super().__init__(node)
        self.n = n
    
    #Protocol definition starts here
    def run(self):
        yield self.await_port_input(self.node.ports['sin'])
        psi = self.node.ports['sin'].rx_input().items[0]
        
        yield self.await_port_input(self.node.ports['vin'])
        x = self.node.ports['vin'].rx_input().items[0]
        
        if x == 0:
            ns.qubits.operate(psi, ns.Z)
        elif x==1:
            ns.qubits.operate(psi, ns.H)
        m,p=ns.qubits.measure(psi)
        
        self.node.ports['vout'].tx_output(m)
        
        yield self.await_port_input(self.node.ports['vin'])
        test = self.node.ports['vin'].rx_input().items[0]
        if(test==0):
            print('Dishonest model-Party')
        else:
            print('Honest model-Party')

In [81]:
class SourceProtocol(NodeProtocol):
    def __init__(self,node,n,honest=True):
        super().__init__(node)
        self.n = n
        self.h=honest
    def run(self):
        qubits= ns.qubits.create_qubits(5)
        if self.h:
            ns.qubits.combine_qubits(qubits)
            ns.qubits.operate(qubits[0], ns.H)
            for i in range(1,self.n):
                ns.qubits.operate([qubits[i], qubits[i+1]], ns.CNOT)
            #print(qubits[0].qstate.qrepr)
        self.node.ports['vout'].tx_output(qubits[0])       
        for i in range(1,self.n):
            self.node.ports['p'+str(i)+'out'].tx_output(qubits[i])
        print("Source- Qubits sent")
        

In [82]:
source=SourceProtocol(source_node,n)
verifier=VerifierProtocol(verifier_node,n)
party=[]
for i in range(1,n):
    party.append(PartyProtocol(parties_node["Party"+str(i)],n))

In [83]:
source.start()
verifier.start()
for i in range(0,n-1):
    party[i].start()
run_stats = ns.sim_run()
source.stop()
verifier.stop()
for i in range(0,n-1):
    party[i].stop()

Source- Qubits sent
True
Honest model
Honest model-Party


In [92]:
source_dh=SourceProtocol(source_node,n,False)
source_dh.start()
verifier.start()
for i in range(0,n-1):
    party[i].start()
run_stats = ns.sim_run()
source_dh.stop()
verifier.stop()
for i in range(0,n-1):
    party[i].stop()

Source- Qubits sent
False
Dishonest model
Dishonest model-Party
